In [ ]:
import numpy as np

# --- 1. 設定と擬似環境 ---

# 真の「最適解」（アルゴリズムには秘密！）
# 例: 前半の層は0.35、後半の層は0.9の比率で混ぜるのが最強、など
TRUE_OPTIMAL_WEIGHTS = np.array([0.35, 0.65, 0.1, 0.9]) 

def mock_evaluate_model(weights):
    """
    【ここが重い処理の代わり】
    実際はここで mergekit を走らせ、ベンチマーク(GSM8k等)を解かせてスコアを返す。
    今回は「正解に近いほど高得点」になる擬似関数を用意。
    """
    # 探索範囲外(0~1を超えた)の異常値をペナルティ化
    # ※Task Arithmeticなどの手法では負の値も許容されますが、
    # 　今回は単純な線形補間(Linear/SLERP)を想定して 0.0〜1.0 に制限します。
    if np.any(weights < 0) or np.any(weights > 1):
        return -100.0
    
    # 正解との距離（二乗誤差）を計算し、スコア化（100点満点）
    mse = np.mean((weights - TRUE_OPTIMAL_WEIGHTS) ** 2)
    score = 100 * np.exp(-10 * mse) 
    
    # 実際のベンチマークのような「ゆらぎ（ノイズ）」を加える
    noise = np.random.normal(0, 0.5)
    return max(0, score + noise)

# --- 2. 進化的アルゴリズムの実装 ---

class SimpleEvolutionaryStrategy:
    def __init__(self, num_params, pop_size=10, elite_size=2):
        self.num_params = num_params
        self.pop_size = pop_size     # 1世代あたりの個体数（レシピの数）
        self.elite_size = elite_size # 次世代に残すエリート数
        
        # 初期値：平均0.5, 分散0.2の正規分布でランダム生成
        self.population = np.random.normal(0.5, 0.2, (pop_size, num_params))

    def step(self, generation):
        """1世代分の進化を実行"""
        scores = []
        
        # 1. 評価フェーズ（個体ごとにベンチマーク実行）
        for i in range(self.pop_size):
            genome = self.population[i]
            score = mock_evaluate_model(genome)
            scores.append(score)
        
        scores = np.array(scores)
        
        # 2. 選択フェーズ（スコア順にソート）
        sorted_indices = np.argsort(scores)[::-1] # 降順
        elites = self.population[sorted_indices[:self.elite_size]]
        top_score = scores[sorted_indices[0]]
        
        # ログ出力
        print(f"Gen {generation}: Best Score = {top_score:.2f} | Params = {np.round(elites[0], 3)}")
        
        # 3. 交叉・変異フェーズ（次世代を作る）
        # ※本来のCMA-ESでは「共分散行列」を更新しますが、
        # 　ここでは簡易的に「エリート個体にノイズを加える（変異）」ことで探索します。
        next_population = []
        next_population.extend(elites) # エリートはそのまま継承
        
        while len(next_population) < self.pop_size:
            # 親を選んで変異（Mutation）させる
            # (ヒント: ここで複数の親を混ぜれば「交叉(Crossover)」になります)
            parent = elites[np.random.randint(0, self.elite_size)]
            mutation = parent + np.random.normal(0, 0.1, self.num_params) # 少し値をずらす
            mutation = np.clip(mutation, 0.0, 1.0)
            next_population.append(mutation)
            
        self.population = np.array(next_population)
        return top_score

# --- 3. メイン実行 ---

if __name__ == "__main__":
    print(f"🧬 進化的マージシミュレーション開始")
    print(f"🎯 目標: 秘密の最適配合 {TRUE_OPTIMAL_WEIGHTS} を探索する\n")
    
    # パラメータ数4, 1世代20個体
    es = SimpleEvolutionaryStrategy(num_params=4, pop_size=20, elite_size=4)
    
    for gen in range(1, 21): # 20世代回す
        best_score = es.step(gen)
        if best_score > 98.0:
            print("\n🎉 最適解を発見しました！")
            break